In [2]:
import arcpy
from collections import defaultdict

# 替换为你的矢量文件路径
feature_class = r"E:\stu1\Projects\POI2023\POI2023.gdb\accuracy_check"


In [3]:
# 打印属性表中所有字段的名称
fields = arcpy.ListFields(feature_class)
for field in fields:
    print(field.name)

OBJECTID
Shape
ORIG_FID
Shape_Leng
Block_ID
FD_A
FD_R
FD_B
FD_M
FD_G
FD_S
KD_R
KD_A
KD_B
KD_G
KD_M
KD_S
KDB_M
KDB_G
KDB_S
KDB_B
KDB_R
KDB_A
DE_FD_A
DE_FD_B
DE_FD_R
DE_FD_M
DE_FD_G
DE_FD_S
DE_KD_A
DE_KD_B
DE_KD_G
DE_KD_M
DE_KD_S
DE_KD_R
DE_KDB_A
DE_KDB_B
DE_KDB_G
DE_KDB_M
DE_KDB_S
DE_KDB_R
FD_A_Prop
FD_B_Prop
FD_R_Prop
FD_G_Prop
FD_S_Prop
FD_M_Prop
KD_A_Prop
KD_B_Prop
KD_R_Prop
KD_G_Prop
KD_S_Prop
KD_M_Prop
KDB_A_Prop
KDB_B_Prop
KDB_R_Prop
KDB_G_Prop
KDB_S_Prop
KDB_M_Prop
DE_FD_A_Pr
DE_FD_B_Pr
DE_FD_R_Pr
DE_FD_G_Pr
DE_FD_S_Pr
DE_FD_M_Pr
DE_KD_A_Pr
DE_KD_B_Pr
DE_KD_R_Pr
DE_KD_G_Pr
DE_KD_S_Pr
DE_KD_M_Pr
DE_KDB_A_P
DE_KDB_B_P
DE_KDB_R_P
DE_KDB_G_P
DE_KDB_S_P
DE_KDB_M_P
MaxLandUse
MaxLandU_1
FD_UrbanFu
FD_Leading
KD_UrbanFu
KD_Leading
KDB_UrbanF
KDB_Leadin
DE_FD_Urba
DE_FD_Lead
DE_KD_Urba
DE_KD_Lead
DE_KDB_Urb
DE_KDB_Lea
FD_Entropy
KD_Entropy
KDB_Entrop
DE_FD_Entr
DE_KD_Entr
DE_KDB_Ent
Shape_Length
Shape_Area


In [7]:
# 初始化一个字典来存储每个街区的结果
results = defaultdict(list)

# 确定可能的方法和类型
methods = ["FD", "KD", "KDB", "DE_FD", "DE_KD", "DE_KDB"]  # 示例：添加所有可能的方法前缀
types = ["A", "R", "M", "B", "G", "S"]
block_id_field = "Block_ID"

In [8]:
# 获取所有字段名
all_fields = [f.name for f in arcpy.ListFields(feature_class)]

# 构建字段名列表
fields = [block_id_field]  # 首先添加街区ID字段
for method in methods:
    for type in types:
        # 搜索匹配的字段名，考虑到名字可能被截断
        field_candidates = [f for f in all_fields if f.startswith(f"{method}_{type}_") and (f.endswith('P') or f.endswith('Pr') or f.endswith('Prop'))]
        if field_candidates:
            assert len(field_candidates) == 1, f"Multiple matching fields for type '{type}' and method '{method}': {field_candidates}"
            fields.append(field_candidates[0])

# 遍历每个街区
with arcpy.da.SearchCursor(feature_class, fields) as cursor:
    for row in cursor:
        block_id = row[0]  # 获取街区ID
        for i, method in enumerate(methods, start=1):
            # 对于每种方法，提取相应类型的比例，并排序
            offset = (i - 1) * len(types)  # 计算当前方法类型比例字段的起始索引
            method_props = {type: row[offset + j] for j, type in enumerate(types)}
            sorted_props = sorted(method_props.items(), key=lambda item: item[1], reverse=True)
            
            # 获取比例最高的两个类型及其比例
            top_two = sorted_props[:2]
            results[block_id].append((method, top_two))

# 输出或处理结果
for block_id, methods_results in results.items():
    print(f"Block ID {block_id}:")
    for method_result in methods_results:
        method, top_two = method_result
        print(f"  Method {method}:")
        for type_prop in top_two:
            print(f"    Type: {type_prop[0]}, Proportion: {type_prop[1]}")


Block ID 29:
  Method FD:
    Type: A, Proportion: 29
    Type: R, Proportion: 0.410731225192
  Method KD:
    Type: B, Proportion: 0.393944834752
    Type: R, Proportion: 0.304343394077
  Method KDB:
    Type: B, Proportion: 0.407672550306
    Type: R, Proportion: 0.297709557357
  Method DE_FD:
    Type: R, Proportion: 0.380085882919
    Type: M, Proportion: 0.284822851635
  Method DE_KD:
    Type: B, Proportion: 0.394826979569
    Type: R, Proportion: 0.325854984518
  Method DE_KDB:
    Type: B, Proportion: 0.395483193443
    Type: R, Proportion: 0.324902441432
Block ID 32:
  Method FD:
    Type: A, Proportion: 32
    Type: R, Proportion: 0.681362980494
  Method KD:
    Type: R, Proportion: 0.573048706235
    Type: G, Proportion: 0.265319235952
  Method KDB:
    Type: R, Proportion: 0.675080742649
    Type: G, Proportion: 0.162985232349
  Method DE_FD:
    Type: R, Proportion: 0.748128143317
    Type: B, Proportion: 0.161885681409
  Method DE_KD:
    Type: R, Proportion: 0.5265101813

  Method DE_KD:
    Type: G, Proportion: 0.33186636267
    Type: B, Proportion: 0.311552329099
  Method DE_KDB:
    Type: G, Proportion: 0.357491913037
    Type: B, Proportion: 0.304537977056
Block ID 1063:
  Method FD:
    Type: A, Proportion: 1063
    Type: G, Proportion: 0.270392689196
  Method KD:
    Type: B, Proportion: 0.461884107784
    Type: G, Proportion: 0.256921402107
  Method KDB:
    Type: G, Proportion: 0.402136415457
    Type: B, Proportion: 0.353313374745
  Method DE_FD:
    Type: S, Proportion: 0.42031017498
    Type: G, Proportion: 0.260811612357
  Method DE_KD:
    Type: B, Proportion: 0.335871128541
    Type: G, Proportion: 0.319425123074
  Method DE_KDB:
    Type: B, Proportion: 0.336311821486
    Type: G, Proportion: 0.32000994163


In [9]:
import arcpy
import numpy as np

# 设置环境
feature_class = r"E:\stu1\Projects\POI2023\POI2023.gdb\accuracy_check"
methods = ["FD", "KD", "KDB", "DE_FD", "DE_KD", "DE_KDB"]  # 示例：添加所有可能的方法前缀
types = ["A", "R", "M", "B", "G", "S"]
truth_fields = ["Truth1", "Truth2"]

**考虑占比最高的一项**

In [13]:

# 获取所有字段名
all_fields = [f.name for f in arcpy.ListFields(feature_class)]

# 构建字段名列表，包括考虑到截断的可能性
fields = []
field_mappings = {}
for method in methods:
    for type_char in types:
        # 搜索匹配的字段名，考虑到名字可能被截断
        field_candidates = [f for f in all_fields if f.startswith(f"{method}_{type_char}_") and (f.endswith('P') or f.endswith('Pr') or f.endswith('Prop'))]
        if field_candidates:
            # 如果存在多个匹配字段，这可能是一个数据问题，你需要解决它
            assert len(field_candidates) == 1, f"Multiple matching fields for type '{type_char}' and method '{method}': {field_candidates}"
            # 保存字段名和实际使用的字段名之间的映射
            field_mappings[f"{method}_{type_char}_Prop"] = field_candidates[0]
            fields.append(field_candidates[0])
fields.extend(truth_fields)

# 初始化混淆矩阵字典
confusion_matrices = {method: np.zeros((len(types), len(types)), dtype=int) for method in methods}
accuracy_records = {method: {'correct': 0, 'total': 0} for method in methods}

# 遍历每个街区，计算混淆矩阵和指标
with arcpy.da.SearchCursor(feature_class, fields) as cursor:
    for row in cursor:
        truth_values = [row[fields.index(truth)] for truth in truth_fields]
        for method in methods:
            # 获取该方法的类型比例，并找出最高的类型
            method_props = {type_char: row[fields.index(field_mappings[f"{method}_{type_char}_Prop"])] for type_char in types}
            classified_type = max(method_props, key=method_props.get)
            
            # 真实值与分类值进行比较
            if classified_type in truth_values:
                # 如果分类正确
                accuracy_records[method]['correct'] += 1
            accuracy_records[method]['total'] += 1
            
            # 更新混淆矩阵
            for truth in truth_values:
                if truth in types:  # 如果真值是有效类型
                    truth_index = types.index(truth)
                    classified_index = types.index(classified_type)
                    confusion_matrices[method][truth_index, classified_index] += 1

# 计算Kappa系数和总体准确度
for method, matrix in confusion_matrices.items():
    # 计算总体准确度
    accuracy = accuracy_records[method]['correct'] / accuracy_records[method]['total']
    # 计算随机准确度
    random_accuracy = np.sum(np.sum(matrix, axis=0) * np.sum(matrix, axis=1)) / np.sum(matrix) ** 2
    # 计算Kappa系数
    kappa = (accuracy - random_accuracy) / (1 - random_accuracy)
    
    # 打印结果
    print(f"Method: {method}")
    print(f"Confusion Matrix:\n{matrix}")
    print(f"Overall Accuracy: {accuracy:.2f}")
    print(f"Kappa Coefficient: {kappa:.2f}")


Method: FD
Confusion Matrix:
[[2 0 2 1 3 0]
 [6 6 6 7 3 0]
 [0 0 0 0 1 0]
 [3 2 3 1 2 0]
 [1 2 2 3 3 0]
 [0 0 1 0 0 0]]
Overall Accuracy: 0.37
Kappa Coefficient: 0.23
Method: KD
Confusion Matrix:
[[ 6  0  2  0  0  0]
 [18  0  7  3  0  0]
 [ 0  0  1  0  0  0]
 [ 6  0  3  2  0  0]
 [ 6  0  4  1  0  0]
 [ 0  0  1  0  0  0]]
Overall Accuracy: 0.30
Kappa Coefficient: 0.22
Method: KDB
Confusion Matrix:
[[ 6  0  1  1  0  0]
 [13  0  5 10  0  0]
 [ 0  0  1  0  0  0]
 [ 3  0  4  4  0  0]
 [ 6  0  2  3  0  0]
 [ 0  0  1  0  0  0]]
Overall Accuracy: 0.37
Kappa Coefficient: 0.28
Method: DE_FD
Confusion Matrix:
[[2 0 2 1 2 1]
 [8 3 6 7 4 0]
 [0 0 0 0 1 0]
 [4 1 3 1 2 0]
 [2 0 1 3 4 1]
 [0 0 0 0 1 0]]
Overall Accuracy: 0.30
Kappa Coefficient: 0.18
Method: DE_KD
Confusion Matrix:
[[ 6  0  1  1  0  0]
 [18  0  6  4  0  0]
 [ 1  0  0  0  0  0]
 [ 7  0  3  1  0  0]
 [ 6  0  3  2  0  0]
 [ 0  0  1  0  0  0]]
Overall Accuracy: 0.23
Kappa Coefficient: 0.14
Method: DE_KDB
Confusion Matrix:
[[ 5  0  2  1  0 

**考虑占比最高的两项**

In [14]:
# ... [之前的代码设置环境、获取字段名列表等]

# 初始化计数器
tp = {method: 0 for method in methods}  # 真阳性
fp = {method: 0 for method in methods}  # 假阳性
fn = {method: 0 for method in methods}  # 假阴性

# 遍历每个街区，统计TP, FP, FN
with arcpy.da.SearchCursor(feature_class, fields) as cursor:
    for row in cursor:
        # 真值
        truth_values = set([row[fields.index(truth)] for truth in truth_fields])
        for method in methods:
            # 提取并排序当前方法的类型比例
            method_proportions = [(row[fields.index(field_mappings[f"{method}_{type_char}_Prop"])], type_char) for type_char in types]
            # 前两高的类型
            top_two_types = set(type_char for _, type_char in sorted(method_proportions, key=lambda x: x[0], reverse=True)[:2])
            
            # 真阳性：识别值和真值有交集
            if truth_values.intersection(top_two_types):
                tp[method] += 1
            # 假阳性和假阴性的统计可以在全局范围内进行，而不是单个街区内
            # ...

# 计算准确度等指标
for method in methods:
    # 计算总体准确度（TP / 总数）
    accuracy = tp[method] / float(len(all_fields))
    # 输出结果
    print(f"Method: {method}, TP: {tp[method]}, Accuracy: {accuracy:.2f}")

# ... [假阳性和假阴性的全局统计和Kappa系数的计算需要额外的逻辑]


Method: FD, TP: 24, Accuracy: 0.24
Method: KD, TP: 15, Accuracy: 0.15
Method: KDB, TP: 16, Accuracy: 0.16
Method: DE_FD, TP: 21, Accuracy: 0.21
Method: DE_KD, TP: 12, Accuracy: 0.12
Method: DE_KDB, TP: 12, Accuracy: 0.12
